### import 部分

In [1]:
import torch
import matplotlib.pyplot as plt
import numpy as np
from ultralytics import YOLO
from PIL import Image
import cv2
import time

# Load the model
model = YOLO('./models/yolov9c.pt')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'CUDA status: {torch.cuda.is_available()}')
model.to(device)

CUDA status: True


YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): RepNCSPELAN4(
        (cv1): Conv(
          (conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Sequential(
          (0): RepCSP(
            (cv1): Conv(
              (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
              

### 单张推理

In [4]:
# source = './Mar22EASON1874.mp4'
# # Run inference on the source
# results = model(source, stream=True)  # generator of Results objects

# # Process results generator (Official suggest)
# for result in results:
#     boxes = result.boxes  # Boxes object for bounding box outputs
#     masks = result.masks  # Masks object for segmentation masks outputs
#     keypoints = result.keypoints  # Keypoints object for pose outputs
#     probs = result.probs  # Probs object for classification outputs
#     result.show()  # display to screen
#     # result.save(filename='result.jpg')  # save to disk

# ------------------------------------------------
results = model('./testfile/bus.jpg')
# Show the results
for r in results:
    im_array = r.plot()  # plot a BGR numpy array of predictions
    image_height, image_width = im_array.shape[:2]
    window_width, window_height = 1280, 720
    scale = min(window_width / image_width, window_height / image_height)
    image = cv2.resize(im_array, None, fx=scale, fy=scale)
    # Create a window that can be resized
    cv2.namedWindow('Image Window', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('Image Window', window_width, window_height)
    
    cv2.imshow('Image Window', im_array)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    # im.save('results.jpg')  # save image
    print('process complete')
# ------------------------------------------------



image 1/1 d:\files\VSCode\HSR-Clockie'sExtras\bus.jpg: 640x480 5 persons, 1 bus, 261.3ms
Speed: 2.5ms preprocess, 261.3ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 480)
process complete


### 视频推理

In [ ]:
capture=cv2.VideoCapture("./testfile/video_360.mp4")#检测的视频路径

width = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

fps = 25

# 创建视频
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('./testfile/1.mp4', fourcc=fourcc,fps=fps, frameSize=(width,height)) #保存检测视频的路径
window_width, window_height = 1280, 720
while(True):
    t1 = time.time()
    # 读取某一帧

    ref,frame=capture.read()
    if not ref:
        capture.release()
        out.release()
        cv2.destroyAllWindows()
        break

    # 格式转变，BGRtoRGB
    frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    # 转变成Image
    frame = Image.fromarray(np.uint8(frame))

    result = model(frame)

    # RGBtoBGR满足opencv显示格式
    # frame = cv2.cvtColor(result[0].plot(), cv2.COLOR_RGB2BGR)
    frame = result[0].plot()

    fps  = ( fps + (1./(time.time()-t1)) ) / 2
    print(f"fps= {fps:.2f}")
    frame = cv2.putText(frame, f"fps= {fps:.2f}", (0, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    image_height, image_width = frame.shape[:2]
    scale = min(window_width / image_width, window_height / image_height)
    image = cv2.resize(frame, None, fx=scale, fy=scale)

    cv2.imshow("video",image) #显示
    cv2.resizeWindow('video', window_width, window_height)
    cv2.namedWindow('video', cv2.WINDOW_NORMAL)
    
    out.write(image) #保存视频
    
 
    c = cv2.waitKey(1) & 0xff 
    if c==27:
        capture.release()
        out.release()
        cv2.destroyAllWindows()
        break

    if cv2.getWindowProperty('video', cv2.WND_PROP_VISIBLE) < 1:
        capture.release()
        cv2.destroyAllWindows()
        out.release()
        break
